<h1>Preprocessing - Design of the Viseu Aerodrome Runway</h1>
<br> In this file you will find information about databases, data preparation, and data processing.

<h2>Basic info</h2>

© Tiago Tamagusko (tamagusko@gmail.com) - <a href="https://tamagusko.github.io">https://tamagusko.github.io</a>
<br> Version: 0.5 (28/12/2019)
<br> Using: Jupyter Notebook 6.0.2, Python 3.8, Linux 4.19.88-1-MANJARO x64, UTF-8.
<br> Required libraries: pandas 0.25.3, sys 3.8.0, geopy 1.20.0.
<br> Project Page: <a href="https://github.com/tamagusko/ViseuAirportStudy">github.com/tamagusko/ViseuAirportStudy</a>
<br> Licence: Apache-2.0, see <a href="/LICENSE.md">LICENSE.md</a> for more details.


<h2>Problem</h2>

Viseu Aerodrome is interested in servicing bigger aircraft, which enable longer range and carry more passengers per aircraft [1]. This requires improving the structural strength of the pavement and increasing the length of its runway.

<h2>Proposal</h2>

In order to design an airport pavement, it is necessary to know two main characteristics, namely the Pavement Classification Number¹ (PCN) and the extension of the runway.

Thus, the proposed challenge is to design a pavement with capacity (PCN) and extension to meet a desired group of routes.

<h2>Project data structure</h2>
    
    ├── 1-preprocessing.ipyng    # Database preprocessing 
    ├── 2-analysis.ipynb               # Data analysis
    ├── data                  
    │          ├── raw                       # Raw data
    │          ├── processed           # Data processed
    ├── reports                              # Outputs
    
<h2>Input data</h2>

1. Aircraft database - <a href="/data/processed/AircraftData.csv">AircraftData.csv</a>
   <br>a. **Aircraft**: Aircraft identification;
   <br>b. **ACN**²: Aircraft Classification Number;
   <br>c. **RequiredExtension**: Runway length required for landing and takeoff operations;
   <br>d. **Autonomy**: Aircraft autonomy;
   <br>e. **Passagers**³: Number of passengers carried.
   <br> **Updated: 25/12/2019**

2. Airport database  - <a href="/data/processed/AirportTestData.csv">AirportTestData.csv</a>  [2] 
   <br>a. **Airport ID**: 	Unique identifier for this airport;
   <br>b. **Airport**: Name of airport;
   <br>c. **City**:  Main city served by airport;
   <br>d. **Country**: 	Country or territory where airport is located;
   <br>e. **IATA**:  3-letter IATA code - **Unused data**;
   <br>f. **ICAO**:  4-letter ICAO code;
   <br>g. **Latitude**: 	Latitutide in decimal degrees. Negative is South, positive is North;
   <br>h. **Longitude** :	Longitude in decimal degrees. Negative is West, positive is East;
   <br>i. **Altitude**: 	Altitude in feet - **Unused data**;
   <br>j. **Timezone**: 	Hours offset from UTC  - **Unused data**;
   <br>k. **DST**: 	Daylight savings time. One of E (Europe), A (US/Canada), S (South America), O (Australia), Z (New Zealand), N (None) or U (Unknown)  - **Unused data**;
   <br>l. **Tz**: Timezone in "tz" (Olson) format, eg. "America/Los_Angeles" - **Unused data**;
   <br>m. **Type**: 	Type of the airport - **Unused data**;
   <br>n. **Source**: 	Source of this data - **Unused data**.
   <br> **Updated: 23/12/2019**
3. Airport Database⁴ with extra data - <a href="/data/processed/AirportTestData.csv">AirportTestExtraData.csv</a> [3]
   <br>a. **ICAO**:  4-letter ICAO code;
   <br>a. **RWYLenght**:  Runway length in meters;
   <br>a. **PCN**:  Runway structural strength number.
   <br> **Updated: 28/12/2019**
   
The data is UTF-8 encoded.

If you want to change the list of **aircraft** under study, edit the file: **Aircraft database (AircraftData.csv)**
<br>If you would like to change the list of **airports** under study, edit the file: **Airport extra database (AirportExtraData.csv)**

Notes: 

**¹** Indicates, among other elements, the structural strength of the pavement. <br>
**²** It is related to the impact of the aircraft on the pavement, and must be less than or equal to the runway PCN. <br>
**³** Information not required. <br>
**⁴** Shengen area airports only (lower airport security requirements).


<h1>1. Data Preparation</h1>

In [ ]:
# Import needed libraries

import pandas as pd
import sys
!{sys.executable} - m pip install geopy
from geopy import distance

# Reference airport, change this value (RefAirportICAO) to choose another airport
RefAirportICAO = "LPVZ"

In [ ]:
# Reading data

# Database developed by the author
aircrafts = pd.read_csv("data/processed/AircraftData.csv", sep=",", index_col ="Aircraft")
# Database adapted from: https://openflights.org/data.html
airports = pd.read_csv("data/processed/AirportData.csv", sep=",", index_col ="ICAO")
# Database developed with data from https://worldaerodata.com
airportsExtra = pd.read_csv("data/processed/AirportExtraData.csv", sep=",", index_col="ICAO")

In [ ]:
# Resulting dataframe - aircrafts: Aircraft,ACN,RequiredExtension,Autonomy,Passagers
aircrafts  # Printing the data to test

In [ ]:
# Importing values from AirportTestExtraData.csv and loading on airports dataframe
for index, row in airportsExtra.iterrows():
    toImport = airportsExtra.loc[[index], ["RWYLenght", "PCN"]]
    toImport = (float(toImport.iloc[:, 0]), float(toImport.iloc[:, 1]))
    # Loading data in airports dataframe
    airports.loc[airports.index == index, ["RWYLenght", "PCN"]] = toImport

# Dropping the unwanted data in airports dataframe
airports = airports.drop("ID", 1)
airports = airports.drop("IATA", 1)
airports = airports.drop("Timezone", 1)
airports = airports.drop("DST", 1)
airports = airports.drop("TZ", 1)
airports = airports.drop("Type", 1)
airports = airports.drop("Source", 1)
# if RWYLenght or PCN are 0: delete row
airports = airports.drop(
    airports[(airports.RWYLenght == 0) | (airports.PCN == 0)].index)

# NOTE: Airports with incomplete or questionable data:
# LIRF/Roma - no values for PCN (29/12/2019)

In [ ]:
# Checking the number of airports with correct data for analysis
airports.shape

In [ ]:
# Resulting dataframe - airports:  ICAO,Airport,City,Country,Latitude,Longitude,Altitude,DistanceToRef(Null),RWYLenght,PCN

# Printing data from the first five lines to test
airports.head(5)

<h1>2. Data processing</h1>

The distance (**DistanceToRef**) in kilometers to the reference airport (LPVZ) shall be calculated.
<br><br>Current Dataframe (airports): ICAO, Airport, City, Country, Latitude, Longitude, Altitude, RWYLenght, PCN.
<br>Required dataframe (airports): ICAO, Airport, City, Country, Latitude, Longitude, Altitude, **DistanceToRef**, RWYLenght, PCN.

In [ ]:
# Saving values to the reference airport
RefAirport = airports.loc[[RefAirportICAO], ["Latitude", "Longitude", "Altitude"]]
RefAirport = (float(RefAirport.iloc[:, 0]), float(RefAirport.iloc[:, 1]), float(RefAirport.iloc[:, 2]))

# Calculating distance from Viseu Airport (LPVZ)
for index, row in airports.iterrows():
    toAirport = airports.loc[[index], ["Latitude", "Longitude", "Altitude"]]
    toAirport = (float(toAirport.iloc[:, 0]), float(
        toAirport.iloc[:, 1]), float(toAirport.iloc[:, 2]))
    # Calculation geodesic distance
    geodesicDistance = distance.geodesic(RefAirport, toAirport).km
    # Loading results into dataframe
    airports.loc[index, "DistanceToRef"] = round(geodesicDistance, 1)

# Resulting dataframe ICAO, Airport, City, Country, Latitude, Longitude, Altitude, DistanceToRef, RWYLenght, PCN.

In [ ]:
# Printing data from the first five lines to test
airports.head(5)

In [ ]:
# Saving the results to a csv file (AirportProcessedData.csv) in the reports folder
airports.to_csv('reports/AirportProcessedData.csv')

<h3> Go to <a href="/2-analysis.ipyng">2-analysis.ipyng</a> to view data analysis. </h3>

<h1> References</h1>

[1] Martins, J. P. F. (2018). Reflexão sobre a viabilidade e localização de uma infraestrutura aeroportuária na região Centro de Portugal (Universidade do Porto). https://doi.org/10.13140/RG.2.2.34944.69124
<br>[2] OpenFlights (2019). Airport, airline and route data. Retrieved December 23, 2019, from https://openflights.org/data.html
<br>[3] World Aero Data (2019). World Aeronautical Database. Retrieved December 28, 2019, from https://worldaerodata.com<br><br>